# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from sympy import *
from PIL import Image
import pickle

In [2]:
%matplotlib tk

# Provided functions

In [3]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.axis('off')
    plt.imshow(im)
    print(f'Set at least {min_lines} lines to compute vanishing point')
    print(f'The delete and backspace keys act like right clicking')
    print(f'The enter key acts like middle clicking')
    while True:
        print('Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print(f'Need at least {min_lines} lines, you have {n} now')
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [4]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [23]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR CODE>
    
    l1 = lines[:,0] / np.sqrt(lines[:,0][0]**2 + lines[:,0][1]**2) 
    l2 = lines[:,1] / np.sqrt(lines[:,1][0]**2 + lines[:,1][1]**2)
    l3 = lines[:,2] / np.sqrt(lines[:,2][0]**2 + lines[:,2][1]**2)
    A = np.vstack((l1,l2,l3))

    u, s, v = np.linalg.svd(A)
    vp = v[-1,:]
    vp /= vp[-1]
    print('vanishing point:', vp)
    return vp


In [24]:
def get_horizon_line(vanishing_points):
    """
    Calculates the ground horizon line.
    """
    # <YOUR CODE>
    
    #The horizional line is the cross of two valishing points x and y
    p1 = vanishing_points[:, 0]
    p2 = vanishing_points[:, 1]
    
    res = np.cross(p1, p2)
    
    
   #The horizional line is the cross of two valishing points x and y
    res = res / np.sqrt((res[0]**2 + res[1]**2))
    
    print('Horizon Line:', res)
    return res


In [25]:
def plot_horizon_line(im, vpts, line):
    """
    Plots the horizon line.
    """
    # <YOUR CODE>
    h,w,_ = im.shape
    a,b,c = line
    y1 = - c/b
    y2 = - (c+w*a)/b
    
    ax.plot([0, w], [y1, y2], 'r')
    ax.imshow(im)
    





In [26]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
#     Since  the fact that the vanishing directions are orthogonal , 
#     Solve for the focal length and optical center (principal point) of the camera. 
    f = Symbol('f')
    px = Symbol('px')
    py = Symbol('py')

    
    K = Matrix(((f,0,px),(0,f,py), (0,0,1)))
    K_inverse = K.inv()
    
    #get the vi for three directions. 
    vpts = Matrix(vpts)
    vp1 = vpts[:,0]
    vp2 = vpts[:,1]
    vp3 = vpts[:,2]
    
#   solve the function that vi.T @ K_inverse.T @ K_inverse @ vj = 0

    K1 = (vp1.T * K_inverse.T * K_inverse * vp2)
    K2 = (vp1.T * K_inverse.T * K_inverse * vp3)
    K3 = (vp2.T * K_inverse.T * K_inverse * vp3)
    
    #solve the results. 
    res = solve([K1,K2,K3],[f,px,py] )
    
    #get the answer
    f = res[0][0]
    px = res[0][1]
    py = res[0][2]
    K = np.array([[f,0,px], [0,f,py], [0,0,1]]).astype(np.float32)
    return f, px, py, K

In [27]:
def get_rotation_matrix(vpts, f, px, py):
    """
    Computes the rotation matrix using the camera parameters.
    """
    #After we got the vanishing points, f, px, py. 
    #We can get that the rotation matrix is the dot product the the inverse of K 
    #and the vanishing points in each direction. 
    vx = vpts[:,1]
    vy = vpts[:,2]
    vz = vpts[:,0]
    K_inv = np.linalg.inv(K)
    
    r1 = K_inv.dot(vx)
    r2 = K_inv.dot(vy)
    r3 = K_inv.dot(vz)
    
#     We also need to normolize our answer at the end. 
    r1 /= np.linalg.norm(r1)
    r2 /= np.linalg.norm(r2)
    r3 /= np.linalg.norm(r3)
    
    R = np.vstack((r1, r2, r3)).T
 
    return R

In [28]:
def get_homography(im, obj):
    """
    Compute homography for transforming the image into fronto-parallel 
    views along the different axes.
    """
    K_inv = np.linalg.inv(K)
    H = K.dot(R).dot(K_inv)

    return H


# Main function

In [29]:
im = np.asarray(Image.open('./eceb.jpg'))

# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('./data.pickle'):
    with open('./data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f)
    num_vpts = 3

In [30]:
# Click and save the line data for vanishing points. This snippet 
# opens up an interface for selecting points and writes them to 
# data.pickle file. The file is over-written.

# num_vpts = 3
# all_n, all_lines, all_centers = [], [], []

# for i in range(num_vpts):
#     print(f'Getting vanishing point {i}')
    
#     # Get at least three lines from user input
#     n_i, lines_i, centers_i = get_input_lines(im)
#     all_n.append(n_i)
#     all_lines.append(lines_i)
#     all_centers.append(centers_i)

# with open('data.pickle', 'wb') as f:
#     pickle.dump([all_n, all_lines, all_centers], f)

In [31]:
# Part (1)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))

for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    # <YOUR CODE> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(all_lines[i])
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    fig.savefig('Q3_vp{:d}.pdf'.format(i), bbox_inches='tight')

vanishing point: [-1.93040643e+03  9.16102077e+01  1.00000000e+00]
vanishing point: [ 3.67634539e+03 -9.95653021e+01  1.00000000e+00]
vanishing point: [2.17464930e+03 5.66480795e+03 1.00000000e+00]


In [33]:
# Part (2) Computing and plotting the horizon
# <YOUR CODE> Get the ground horizon line
horizon_line = get_horizon_line(vpts)

# <YOUR CODE> Plot the ground horizon line
fig = plt.figure(); ax = fig.gca()
plot_horizon_line(im,vpts,horizon_line)
fig.savefig('Q3_horizon.pdf', bbox_inches='tight')

Horizon Line: [  0.03407757   0.99941919 -25.77344097]


In [34]:
# Part (3) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
f, px, py, K = get_camera_parameters(vpts)
print("f is:", f)
print("px is:", px)
print("py is:", py)
print("K is:", K)


f is: -2299.61367284199
px is: 2019.70603363198
py is: 1120.66717441262
K is: [[-2.2996138e+03  0.0000000e+00  2.0197061e+03]
 [ 0.0000000e+00 -2.2996138e+03  1.1206671e+03]
 [ 0.0000000e+00  0.0000000e+00  1.0000000e+00]]


In [35]:
# Part (4) Computing Rotation Matrices
# <YOUR CODE> Solve for the rotation matrix
R = get_rotation_matrix(vpts, f, px, py)
print("Rotation matirx is :", R)

Rotation matirx is : [[-0.53687352 -0.03040942  0.84311451]
 [ 0.39544545 -0.8918408   0.21964255]
 [ 0.74524474  0.4513261   0.49083101]]
